In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load the geometry and experimental data files
# geom_df1 = pd.read_csv('volume1_geom.csv') 
# exp_df1 = pd.read_csv('volume1_exp.csv')   
# geom_df2 = pd.read_csv('volume1_geom.csv') 
# exp_df2 = pd.read_csv('volume1_exp.csv')   
# geom_df3 = pd.read_csv('volume1_geom.csv') 
# exp_df3 = pd.read_csv('volume1_exp.csv')    

geom_df = pd.read_csv('volume1_geom.csv') 
exp_df = pd.read_csv('volume1_exp.csv')   
# geom_df = pd.concat([geom_df1, geom_df2, geom_df3], ignore_index=True)
# exp_df = pd.concat([exp_df1, exp_df2, exp_df3], ignore_index=True)
# Merge the geometry and experimental data on the 'BladeName' column
# merged_df = pd.merge(exp_df, geom_df, on='BladeName')

# Group the geometric parameters (c/R, r/R, beta) by BladeName
geometry_grouped = geom_df.groupby(['BladeName', 'D', 'P', 'Family']).agg({
    'c/R': list,
    'r/R': list,
    'beta': list
}).reset_index()

# Merge the grouped geometric data back with the experimental data
final_df = pd.merge(exp_df, geometry_grouped, on=['BladeName', 'D', 'P', 'Family'])

# Extract scalar and array features
X_scalar = final_df[['B', 'D', 'P', 'J', 'N']]
X_array = final_df[['c/R', 'r/R', 'beta']].applymap(np.array)  # Convert lists to arrays
y = final_df['eta']  # Target variable

# Determine the maximum lengths of array features for padding
max_len_c = max(len(arr) for arr in X_array['c/R'])
max_len_r = max(len(arr) for arr in X_array['r/R'])
max_len_beta = max(len(arr) for arr in X_array['beta'])

# Function to pad arrays with zeros to ensure uniform length
def pad_array(array, max_len):
    return np.pad(array, (0, max_len - len(array)), mode='constant')

# Pad each array feature separately
X_c_padded = np.array([pad_array(arr, max_len_c) for arr in X_array['c/R']])
X_r_padded = np.array([pad_array(arr, max_len_r) for arr in X_array['r/R']])
X_beta_padded = np.array([pad_array(arr, max_len_beta) for arr in X_array['beta']])

# Combine scalar and array features into a single feature matrix
X_combined = np.hstack([X_scalar, X_r_padded, X_c_padded, X_beta_padded])

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.1, random_state=42)

# Display the shapes of the training and test sets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


C:\Users\Piotr\AppData\Local\Temp\ipykernel_38112\1040886736.py:32: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_array = final_df[['c/R', 'r/R', 'beta']].applymap(np.array)  # Convert lists to arrays


X_train shape: (8567, 59)
X_test shape: (952, 59)
y_train shape: (8567,)
y_test shape: (952,)


In [67]:
X_train.shape

(8567, 59)

In [57]:
geometry_grouped['r/R'][0]

[0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9,
 0.95,
 1.0]

In [58]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

# Define the Decision Tree Regressor model
dt_model = DecisionTreeRegressor(random_state=42)

# Train the model on the training set
dt_model.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = dt_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Test Mean Absolute Error: {mae:.4f}')
print(f'Test Root Mean Squared Error: {rmse:.4f}')
print(f'R2 squared: {r2:.4f}')


Test Mean Absolute Error: 0.0602
Test Root Mean Squared Error: 0.2403
R2 squared: 0.8220


In [60]:
import pickle
# Save model
filename = "model_new_J.sav"
pickle.dump(dt_model, open(filename, 'wb'))